In [ ]:
import pandas as pd
import math
import nltk
import random
import matplotlib.pyplot as plt
import numpy as np
import re
import pickle
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from nltk.classify.textcat import TextCat
import dask.dataframe as dd
import multiprocessing
import random
from scipy.sparse.linalg import cg

nltk.download('punkt')
nltk.download('crubadan')
nltk.download('stopwords')
tokenizer = RegexpTokenizer('\w+')

nltk.download('own-1.4')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package crubadan to /root/nltk_data...
[nltk_data]   Package crubadan is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading own-1.4: Package 'own-1.4' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
users = pd.read_csv("/content/drive/MyDrive/content_based/final_users.csv")
users.head()

,ID,Name,Rating,Name_ID,New_ID
0,1,Siddhartha,5,1,1
1,1,"The Hunger Games (The Hunger Games, #1)",5,2,1
2,1,The Authoritative Calvin and Hobbes: A Calvin ...,5,3,1
3,1,The Return of the Indian (The Indian in the Cu...,5,4,1
4,1,Dark Apprentice (Star Wars: The Jedi Academy T...,3,5,1


In [ ]:
books = pd.read_csv("/content/drive/MyDrive/content_based/final_books.csv")
books.head()

,Name,Authors,Rating,PublishYear,Publisher,CountsOfReview,Language,PagesNumber,Description
0,The Sleeping Beauty,Jacob Grimm,3.84,1995,NorthSouth,0,NaN,32,Enraged at not being invited to the princess' ...
1,Threshold,Bill Myers,3.83,1997,Zondervan Publishing Company,3,NaN,331,Some say Brandon Martus has a mysterious abili...
2,What Is the What,Dave Eggers,4.16,2007,Vintage Canada,65,NaN,538,<b>What Is the What</b> is the story of Valent...
3,"Parasyte, Volume 2",Hitoshi Iwaaki,4.29,2007,Del Rey,77,eng,288,<b>PERSONAL DEMON</b><br /><br />Shin looks li...
4,"Parasyte, Volume 3",Hitoshi Iwaaki,4.27,2008,Del Rey,48,NaN,283,ALIEN NATION <br />Aliens parasites have invad...


In [ ]:
# Find number of NaN in each column
books.isna().sum()

Name                 0
Authors              0
Rating               0
PublishYear          0
Publisher           73
CountsOfReview       0
Language          7582
PagesNumber          0
Description          0
dtype: int64

In [ ]:
books[["Publisher"]] = books[["Publisher"]].fillna("unknown")

In [ ]:
books.Language.unique()

array([nan, 'eng', 'en-US', 'spa', 'fre', 'en-GB', 'per', 'ind', 'ger',
       'por', 'swe', 'ita', 'jpn', 'mul', 'en-CA', 'zho', 'nor', 'pol',
       'elx', 'grc'], dtype=object)

In [ ]:
html_pattern = re.compile('<[^>]*>')
def remove_html_tags(text):
    return re.sub(html_pattern, r'', text)

books.Description = books.Description.apply(remove_html_tags)
books.head()

,Name,Authors,Rating,PublishYear,Publisher,CountsOfReview,Language,PagesNumber,Description
0,The Sleeping Beauty,Jacob Grimm,3.84,1995,NorthSouth,0,NaN,32,Enraged at not being invited to the princess' ...
1,Threshold,Bill Myers,3.83,1997,Zondervan Publishing Company,3,NaN,331,Some say Brandon Martus has a mysterious abili...
2,What Is the What,Dave Eggers,4.16,2007,Vintage Canada,65,NaN,538,What Is the What is the story of Valentino Ach...
3,"Parasyte, Volume 2",Hitoshi Iwaaki,4.29,2007,Del Rey,77,eng,288,PERSONAL DEMONShin looks like an ordinary high...
4,"Parasyte, Volume 3",Hitoshi Iwaaki,4.27,2008,Del Rey,48,NaN,283,ALIEN NATION Aliens parasites have invaded Ear...


In [ ]:
text = list(books["Description"])
prepro_test = []

for i in text:

  s = ""
  i = i.lower()
  ans = tokenizer.tokenize(i)

  for j in ans:
    
    y = lemmatizer.lemmatize(j)

    if y not in stop_words:
      s = s + " " + y 

  prepro_test.append(s)

for i in range(len(text)):
  books["Description"][i] = prepro_test[i]

books.head()

<ipython-input-9-9bbeafa4294a>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books["Description"][i] = prepro_test[i]


,Name,Authors,Rating,PublishYear,Publisher,CountsOfReview,Language,PagesNumber,Description
0,The Sleeping Beauty,Jacob Grimm,3.84,1995,NorthSouth,0,NaN,32,enraged invited princess christening wicked f...
1,Threshold,Bill Myers,3.83,1997,Zondervan Publishing Company,3,NaN,331,say brandon martus ha mysterious ability see ...
2,What Is the What,Dave Eggers,4.16,2007,Vintage Canada,65,NaN,538,story valentino achak deng refugee war ravage...
3,"Parasyte, Volume 2",Hitoshi Iwaaki,4.29,2007,Del Rey,77,eng,288,personal demonshin look like ordinary high sc...
4,"Parasyte, Volume 3",Hitoshi Iwaaki,4.27,2008,Del Rey,48,NaN,283,alien nation alien parasite invaded earth tak...


In [ ]:
tfidf = TfidfVectorizer(stop_words="english")

# fit and transform the text data`
tfidf_matrix = tfidf.fit_transform(books["Description"])

In [ ]:
print(tfidf_matrix)

  (0, 59691)	0.10788145055827462
  (0, 49661)	0.2561244235575753
  (0, 16291)	0.30931767573901536
  (0, 50573)	0.2625880408530725
  (0, 9107)	0.21967714545662567
  (0, 19996)	0.25347893207066463
  (0, 58845)	0.275843086011429
  (0, 10093)	0.3847221530528204
  (0, 42221)	0.4713187456163782
  (0, 28410)	0.29645024511236756
  (0, 18318)	0.33391271370368547
  (1, 55299)	0.08221155808102343
  (1, 4201)	0.11117774853319748
  (1, 15647)	0.09008190592518951
  (1, 20791)	0.08969954423633458
  (1, 12586)	0.18304724140435863
  (1, 31189)	0.16051428650025865
  (1, 18068)	0.10312015160968924
  (1, 52204)	0.12650339944189656
  (1, 23775)	0.12292734531394457
  (1, 26243)	0.07369594143825216
  (1, 33890)	0.13915367235252044
  (1, 54946)	0.13083742913330101
  (1, 58419)	0.06260787739339835
  (1, 58995)	0.11702687684828343
  :	:
  (11054, 17992)	0.06796493416957619
  (11054, 51787)	0.23356582931193276
  (11055, 16058)	0.559429905388364
  (11055, 19413)	0.31181561850269124
  (11055, 21684)	0.399169378608

In [ ]:
# compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
book_id_to_name = {}

for i in range(len(users['Name_ID'])):
  if users['Name_ID'][i] not in book_id_to_name.keys():
    book_id_to_name[users['Name_ID'][i]] = users['Name'][i]

In [ ]:
print(book_id_to_name)

{1: 'Siddhartha', 2: 'The Hunger Games (The Hunger Games, #1)', 3: 'The Authoritative Calvin and Hobbes: A Calvin and Hobbes Treasury', 4: 'The Return of the Indian (The Indian in the Cupboard, #2)', 5: 'Dark Apprentice (Star Wars: The Jedi Academy Trilogy, #2)', 6: 'Angels & Demons (Robert Langdon, #1)', 7: 'The Return of the King (The Lord of the Rings, #3)', 8: 'The Girl with the Dragon Tattoo (Millennium, #1)', 9: 'Thunderball (James Bond, #9)', 10: 'The BFG', 11: 'The Golden Compass (His Dark Materials, #1)', 12: 'To Kill a Mockingbird', 13: 'Babar the King', 14: 'The Religion (Tannhauser, #1)', 15: 'The Circle', 16: 'The Godfather', 17: 'Where the Red Fern Grows', 18: 'The Anubis Gates', 19: 'Pride and Prejudice', 20: 'The Amber Spyglass (His Dark Materials, #3)', 21: 'Frankenstein', 22: 'The Complete Works', 23: 'The Scarlet Letter', 24: 'Streets of Laredo', 25: 'Killing Rommel', 26: 'Stranger in a Strange Land', 27: 'Think and Grow Rich', 28: 'God Emperor of Dune (Dune Chronicl

In [ ]:
user_book_to_books_id = {}
books_id = {}

for i in range(len(books["Name"])):
  if books["Name"][i] not in books_id.keys():
    books_id[books["Name"][i]] = i

for i in range(len(users["Name_ID"])):
  if users["Name_ID"][i] not in user_book_to_books_id.keys():
    user_book_to_books_id[users["Name_ID"][i]] = books_id[users["Name"][i]]

print(user_book_to_books_id)

book_id_user_books_id = {}
for i in user_book_to_books_id.keys():
  book_id_user_books_id[user_book_to_books_id[i]] = i

print(user_book_to_books_id[11057])
print(book_id_user_books_id)

{1: 139, 2: 9265, 3: 10911, 4: 2746, 5: 8544, 6: 7944, 7: 811, 8: 6225, 9: 9346, 10: 5069, 11: 1323, 12: 3209, 13: 1659, 14: 10190, 15: 10936, 16: 4785, 17: 3480, 18: 3108, 19: 396, 20: 6591, 21: 150, 22: 1137, 23: 100, 24: 1707, 25: 8643, 26: 9772, 27: 2277, 28: 5602, 29: 6329, 30: 5220, 31: 10776, 32: 9307, 33: 7581, 34: 1901, 35: 2694, 36: 2641, 37: 1785, 38: 6277, 39: 4106, 40: 5769, 41: 2602, 42: 1945, 43: 8190, 44: 200, 45: 2206, 46: 1905, 47: 1207, 48: 9516, 49: 6708, 50: 681, 51: 10217, 52: 3733, 53: 1818, 54: 6322, 55: 2826, 56: 2618, 57: 1824, 58: 1571, 59: 8189, 60: 152, 61: 7693, 62: 2905, 63: 8543, 64: 2931, 65: 4159, 66: 3882, 67: 6129, 68: 1913, 69: 1821, 70: 1768, 71: 1582, 72: 532, 73: 6254, 74: 810, 75: 1716, 76: 10116, 77: 7074, 78: 2041, 79: 10228, 80: 7568, 81: 8200, 82: 2894, 83: 5862, 84: 8042, 85: 3230, 86: 1374, 87: 1443, 88: 64, 89: 2710, 90: 7572, 91: 6934, 92: 7519, 93: 8857, 94: 4345, 95: 9532, 96: 896, 97: 8191, 98: 7662, 99: 780, 100: 9260, 101: 1098, 102

In [ ]:
def get_recommendations(book_id, cosine_sim=cosine_sim, books=books):

  # get the cosine similarity scores of the book with all other books
  sim_scores = list(enumerate(cosine_sim[book_id]))
  # print(sim_scores)

  # sort the books based on the cosine similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # get the top 5 most similar books
  sim_scores = sim_scores[1:6]

  return sim_scores

In [ ]:
# userid = 20
def find_already_read_books(userid):
  already_read_books_n = []
  already_read_books = []
  ratings = []
  for i in range(len(users["ID"])):
    if users["New_ID"][i]==userid:         
      # if users["Rating"][i]>=3:   
      ratings.append(users["Rating"][i])                      #will find similar books to only those previously read books which are not disliked by the user
      already_read_books_n.append((user_book_to_books_id[users["Name_ID"][i]], users["Rating"][i]))
  
  minrating = min(ratings)
  # print(minrating)
  maxrating = max(ratings)
  # print(maxrating)
  avg = math.floor((minrating + maxrating)/2)
  # print(avg)
  # print(already_read_books_n)

  for i in range(len(already_read_books_n)):
    if already_read_books_n[i][1] >= avg:
      already_read_books.append(already_read_books_n[i][0])

  # print(already_read_books)
  return already_read_books

In [ ]:
def get_content_recommendations(userid):
  all_recomm_books = []
  already_read_books = find_already_read_books(userid)
  # print(already_read_books)
  # for i in already_read_books:
  #   print(book_id_to_name[book_id_user_books_id[i]])
  books = []
  for i in range(len(already_read_books)):
    sim_scores = get_recommendations(already_read_books[i])

    for i in sim_scores:
      if i not in all_recomm_books and i[0] not in already_read_books and i[0] not in books:
        all_recomm_books.append(i)
        books.append(i[0])

  all_recomm_books = sorted(all_recomm_books, key=lambda x: x[1], reverse=True)

  # print(all_recomm_books)

  final_recomm_books = [i[0] for i in all_recomm_books[:10]]

  # print("Recommended books using Content based Filtering are: -\n")
  # for i in final_recomm_books:
  #   print(book_id_to_name[book_id_user_books_id[i]])
  
  return all_recomm_books, final_recomm_books

In [ ]:
userid = 31
print(get_content_recommendations(userid)[1])

[9635, 6477, 8054, 10303, 4535]


In [ ]:
# all_users_cb_recommendations = []
# for i in range(1,3535):
#   all_


In [ ]:
ffile = open("/content/drive/MyDrive/content_based/f70.pkl","rb")
cf_ratings_mat = pickle.load(ffile)
ffile.close()

In [ ]:
for i in range(10):
  print(cf_ratings_mat[i])

[4.60996879 3.83047743 5.28152522 ... 4.8918093  4.62225343 3.8474758 ]
[3.64095239 4.8555123  4.97809635 ... 4.30031683 4.07950517 4.76757791]
[4.72304556 5.06747926 5.19242865 ... 4.96201728 5.32830976 5.02823689]
[4.88567499 3.97575352 5.09804588 ... 4.23361185 4.481005   4.04105557]
[4.66048348 4.96046806 5.47525815 ... 4.92975044 5.27296751 4.64638874]
[3.90376078 4.79553001 4.96881633 ... 5.06626213 4.98630531 5.31340275]
[3.19044427 4.86363725 5.19273424 ... 4.87642849 4.85803164 4.29301613]
[4.51196217 4.57945785 5.19108666 ... 5.01157607 4.96784749 4.89441642]
[3.68504099 5.21015293 4.8000635  ... 4.98242621 4.83777532 4.63212736]
[4.53284157 4.91172921 5.19096554 ... 3.95847162 4.07552992 4.78357565]


In [ ]:
def min_max_rating(userid):
  minrating = min(cf_ratings_mat[userid])
  maxrating = max(cf_ratings_mat[userid])
  return minrating, maxrating

In [ ]:
accuracy_for_all_users = []
specificity_for_all_users = []

def user_accuracy(userid):

  ratings_of_recomm_books = {}
  all_recomm_books = get_content_recommendations(userid)[0]

  for i in range(len(all_recomm_books)):
    bookid = all_recomm_books[i][0]
    users_bookid = book_id_user_books_id[bookid]
    rating = cf_ratings_mat[userid-1][users_bookid-1]
    ratings_of_recomm_books[users_bookid] = rating
  
  minrating, maxrating = min_max_rating(userid-1)
  user_avg_rating = (minrating+maxrating)/2

  relcount = 0        #no of relevant books 
  nonrelcount = 0     #no of non-relevant books 

  for i in ratings_of_recomm_books.values():
    if i>=user_avg_rating:
      relcount+=1
    else:
      nonrelcount+=1

  tn = 0
  tp = relcount
  fn = 0
  fp = nonrelcount

  for i in range(11057):
    if user_book_to_books_id[i+1] not in all_recomm_books[:][0] and cf_ratings_mat[userid-1][i]<user_avg_rating:
      tn+=1
    elif user_book_to_books_id[i+1] not in all_recomm_books[:][0] and cf_ratings_mat[userid-1][i]>=user_avg_rating:
      fn+=1   

  accuracy = (relcount + tn)/(relcount + tn + nonrelcount + fn)

  specificity = tn / (tn + fp)

  accuracy_for_all_users.append(accuracy)
  specificity_for_all_users.append(specificity)

In [ ]:
for i in range(1,3535):
  print(i)
  user_accuracy(i)


In [ ]:
# file = open("/content/drive/MyDrive/content_based/evaluations/specificity","wb")
# pickle.dump(specificity_for_all_users, file)
# file.close()

# file = open("/content/drive/MyDrive/content_based/cb_users_accuracy","wb")
# pickle.dump(accuracy_for_all_users, file)
# file.close()

In [ ]:
file = open("/content/drive/MyDrive/content_based/cb_users_accuracy","rb")
accuracy_for_all_users = pickle.load(file)
file.close()

file = open("/content/drive/MyDrive/content_based/evaluations/specificity","rb")
specificity_for_all_users = pickle.load(file)
file.close()

In [ ]:
sum_acc = sum(accuracy_for_all_users)
avg_accuracy = sum_acc/len(accuracy_for_all_users)
print(avg_accuracy)

sum_specificity = sum(specificity_for_all_users)
avg_specificity = sum_specificity/len(specificity_for_all_users)
print(avg_specificity)

0.9637632526920618
0.9911863247440367


Recommendations of collaborative filtering

In [ ]:
def printcfbooks(cf_books):
  print("\nRecommended books using Collaborative based Filtering are: -\n")
  for i in range(10):
    print(book_id_to_name[book_id_user_books_id[cf_books[i]]])


def printcbbooks(cb_books):
  print("\nRecommended books using Content based Filtering are: -\n")
  for i in cb_books:
    print(book_id_to_name[book_id_user_books_id[i]])
  
  

In [ ]:
def cf_recomm(userid):
  cf_recomm_all = []
  minrating, maxrating = min_max_rating(userid-1)
  avg_rating = (minrating + maxrating)/2
  already_read_books = find_already_read_books(userid)

  for i in range(len(cf_ratings_mat[userid-1])):
    if cf_ratings_mat[userid-1][i] >= avg_rating and user_book_to_books_id[i+1] not in already_read_books:
      cf_recomm_all.append([user_book_to_books_id[i+1], cf_ratings_mat[userid-1][i]])

  cf_recomm_all = sorted(cf_recomm_all, key=lambda x: x[1], reverse=True)  
  cf_recomm_books = [i[0] for i in cf_recomm_all[:]]

  return cf_recomm_books

In [ ]:
def get_hybrid_recommendations(userid):

  cf_recommendations = cf_recomm(userid)
  printcfbooks(cf_recommendations)

  cb_recommendations = get_content_recommendations(userid)[1]
  printcbbooks(cb_recommendations)

  book_recommendations = []
  for i in cb_recommendations:
    if i in cf_recommendations:
      book_recommendations.append(i)

  if len(book_recommendations) < 10:
    num = 10-len(book_recommendations)

    cf_ratio = math.floor(num*(1/3))
    cb_ratio = num-cf_ratio

    if len(cb_recommendations)-len(book_recommendations) < cb_ratio:
      cf_ratio = cf_ratio + cb_ratio-len(cb_recommendations) + len(book_recommendations)
      cb_ratio = len(cb_recommendations) - len(book_recommendations)

    j = cb_ratio
    k=0
    while j>0:
      if cb_recommendations[k] not in book_recommendations:
        book_recommendations.append(cb_recommendations[k])
        j-=1
      k+=1

    i=cf_ratio
    k=0
    while i>0:
      if cf_recommendations[k] not in book_recommendations:
        book_recommendations.append(cf_recommendations[k])
        i-=1
      k+=1

  # fin_books = []
    
  # print(book_recommendations)

  print("\nRecommended books using Hybird Filtering are: -\n")
  
  for i in book_recommendations:
    print(book_id_to_name[book_id_user_books_id[i]])
                     
  # for i in book_recommendations:
  #   fin_books.append(book_id_to_name[book_id_user_books_id[i]])
    
  # return fin_books

In [ ]:
userid = 31

get_hybrid_recommendations(userid)


Recommended books using Collaborative based Filtering are: -

The Sopping Thursday
Sleepless
Creative Genius: An Innovation Guide for Business Leaders, Border Crossers and Game Changers
Property and Freedom
The New Deal for Artists
Fatal Tide
Sophie's Choice
True Detective (Nathan Heller, #1)
With This Ring
How to Twist a Dragon's Tale (How to Train Your Dragon, #5)

Recommended books using Content based Filtering are: -

The Sorcerer's Companion: A Guide to the Magical World of Harry Potter
Angels Flight (Harry Bosch, #6)
Harry the Dirty Dog
No Roses for Harry!
Selections from Harry Potter and the Order of the Phoenix: Piano Solos

Recommended books using Hybird Filtering are: -

The Sorcerer's Companion: A Guide to the Magical World of Harry Potter
Angels Flight (Harry Bosch, #6)
Harry the Dirty Dog
Selections from Harry Potter and the Order of the Phoenix: Piano Solos
No Roses for Harry!
The Sopping Thursday
Sleepless
Creative Genius: An Innovation Guide for Business Leaders, Borde

In [ ]:
# recc_for_all_users = []

# for i in range(1, 3535):

#   recc_for_all_users.append(get_hybrid_recommendations(i))
#   print(i)

In [ ]:
# print(recc_for_all_users)

[['The Jungle', 'Bridge to Terabithia', 'Notes from Underground', 'The Girl with the Dragon Tattoo (Millennium #1)', 'The Catcher In The Rye', 'Rumble Fish', "The Midwife's Apprentice", 'The House on Mango Street', 'The Art of War', 'Persuasion'], ['God Emperor of Dune (Dune Chronicles, #4)', 'The Lost Symbol (Robert Langdon, #3)', 'Killer in the Rain', 'Casino Royale', 'Austenland (Austenland, #1)', 'From Russia With Love', 'Paul of Dune (Heroes of Dune #1)', 'Growing Up', 'The Great Divorce', 'Bridge to Terabithia'], ['Kiss My Book', 'A Ship Made Of Paper', 'Trivia Queen, 3rd Grade Supreme (Ruby and the Booker Boys, #2)', 'X in Flight (XYZ, #1)', "Max's Dragon Shirt", 'jQuery in Action', 'The Korean War', 'The Godfather', 'The Call of the Wild', 'As I Lay Dying'], ['The Jungle', 'The Garden Party and Other Stories', 'The People of the Mist', 'Autobiography', 'The Confidence-Man: His Masquerade', 'The Railway Children', 'Las Aventuras de Tom Sawyer', 'Sister Carrie', 'Howl and Other P

In [ ]:
# file = open("/content/drive/MyDrive/content_based/user_recommendations","wb")
# pickle.dump(recc_for_all_users, file)
# file.close()

In [ ]:
ffile = open("/content/drive/MyDrive/content_based/user_recommendations","rb")
all_recomm = pickle.load(ffile)
ffile.close()

In [ ]:
print(all_recomm[30])

["The Sorcerer's Companion: A Guide to the Magical World of Harry Potter", 'Angels Flight (Harry Bosch, #6)', 'Harry the Dirty Dog', 'No Roses for Harry!', 'Selections from Harry Potter and the Order of the Phoenix: Piano Solos', 'The Catcher in the Rye', 'The Great Gatsby', 'The Tipping Point: How Little Things Can Make a Big Difference', 'To Kill a Mockingbird', '1984']


-------------


Recommendations for User's Fev Book

In [ ]:
def find_fev_books(userid):
  
  already_read_books = []
  fev_book_names = []
  ratings = []

  for i in range(len(users["ID"])):

    if users["New_ID"][i]==userid:         

      ratings.append(users["Rating"][i])  #will find similar books to only those previously read books which are not disliked by the user
      already_read_books.append((user_book_to_books_id[users["Name_ID"][i]], users["Rating"][i]))
  
  max_rat = max(ratings)

  fev_books = []

  for i in already_read_books:

    if i[1] == max_rat and i[0] not in fev_books:

      fev_books.append(i[0])
      fev_book_names.append(book_id_to_name[book_id_user_books_id[i[0]]])
  
  return fev_books[:3], fev_book_names[:3]

In [ ]:
def sortt(lst):

  return sorted(lst, key=lambda x: x[1], reverse=True)

In [ ]:
def fev_recc(userid):

  all_recomm_books = []

  fev_books, fev_book_names = find_fev_books(userid)
  
  for i in range(len(fev_books)):

    new_books = []
    books = []

    sim_scores = get_recommendations(fev_books[i])

    for i in sim_scores:

      if i not in new_books and i[0] not in fev_books and i[0] not in books:

        new_books.append(i)
        books.append(i[0])
    
    all_recomm_books.append(new_books)
  
  sorted_books = []

  for i in all_recomm_books:

    sorted_books.append(sortt(i))

  # all_recomm_books = sorted(all_recomm_books, key=lambda x: x[1], reverse=True)

  final_recomm_books = []

  for i in sorted_books:

    l = []

    for j in i:

      l.append(j[0])
    
    final_recomm_books.append(l)
  
  fev_recomm = []
  
  for i in final_recomm_books:

    l = []

    for j in i:

      l.append(book_id_to_name[book_id_user_books_id[j]])
    
    fev_recomm.append(l)
    
  return fev_recomm, fev_book_names

In [ ]:
fev, fev_book_names = fev_recc(31)
print(fev_book_names)
print()
for i in fev:
  for j in i:
    print(j)
  print('\n')

['Harry Potter and the Half-Blood Prince (Harry Potter, #6)', 'Harry Potter and the Goblet of Fire (Harry Potter, #4)', 'Harry Potter and the Deathly Hallows (Harry Potter, #7)']

Harry Potter and the Chamber of Secrets (Harry Potter, #2)
No Roses for Harry!
Selections from Harry Potter and the Order of the Phoenix: Piano Solos


Harry Potter and the Chamber of Secrets (Harry Potter, #2)
The Sorcerer's Companion: A Guide to the Magical World of Harry Potter
Angels Flight (Harry Bosch, #6)
Harry the Dirty Dog


No Roses for Harry!
The Sorcerer's Companion: A Guide to the Magical World of Harry Potter
Harry Potter and the Chamber of Secrets (Harry Potter, #2)




In [ ]:
recentyear = max(books["PublishYear"])
print(recentyear)
count = 0
title = []
for i in range(len(books["PublishYear"])):
  if books["PublishYear"][i]>=2018:
    title.append(books["Name"][i])
    count+=1

print(count)
print(title)

2020
8
['The Collected Poems', 'This Is Real and You Are Completely Unprepared: The Days of Awe as a Journey of Transformation', 'Bottomless Belly Button', 'Genesis', "Aaron's Hair", 'Loser', 'Barracoon: The Story of the Last "Black Cargo"', 'Creative Visualization']
